In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
import tensorflow as tf

# Graphic output
from IPython.display import SVG
from keras.models import load_model
from IPython.display import Image
from IPython.display import clear_output

# GPU status detection

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Auxiliary function

In [ ]:
#PlotProgress subclasses the Callback class to plot graph after each epoch
class PlotProgress(tf.keras.callbacks.Callback):

    def __init__(self, entity='loss'):
        self.entity = entity
        
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.fig = plt.figure()
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('{}'.format(self.entity)))
        self.val_losses.append(logs.get('val_{}'.format(self.entity)))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="{}".format(self.entity))
        plt.plot(self.x, self.val_losses, label="val_{}".format(self.entity))
        plt.legend()
        plt.show();


# Dataloader class

In [ ]:
# data_loader
# -input:  csv files path
# -output: absorption coeffcients for each surfaces in 63Hz,125Hz and 250Hz, 500Hz octave band // room size // Transfer functions
class DataLoader(object):
    def __init__(self,path,ratio):
        self.path = path
        self.ratio = ratio
        self.data = []
    # dataset length
    def __len__(self):
        return len(self.data)
    
    #  return one data
    def __get__(self,idx):
        return self.data[idx]
    
    def load_data(self):
        # Get all csvs in your dir.
        files = Path(path).glob('*.csv')
        df_list = []

        # Put all the csv file togther
        for file in files:
            df_list.append(pd.read_csv(file))
        self.data = pd.concat(df_list)
        # Data perturbation by row, keep the order
        self.data = self.data.sample(frac=1).reset_index(drop=True)
    
        # Split absorption coeffcient, room size and transfer functions
        dfs = np.split(self.data, [4,8,12,16,20,24,27], axis=1) 
        self.ac1 = dfs[0]
        self.ac2 = dfs[1]
        self.ac3 = dfs[2]
        self.ac4 = dfs[3]
        self.ac5 = dfs[4]
        self.ac6 = dfs[5]
        self.RoomGeo = dfs[6]
        self.TransFun = dfs[7]
        
        return self.split_data()
    
    def split_data(self):
        # Testing to training rario
        Ratio = self.ratio

        ## Lable split
        ac1_63 = self.ac1['1-63']
        ac1_125 = self.ac1['1-125']
        ac1_250 = self.ac1['1-250']
        ac1_500 = self.ac1['1-500']

        ac2_63 = self.ac2['2-63']
        ac2_125 = self.ac2['2-125']
        ac2_250 = self.ac2['2-250']
        ac2_500 = self.ac2['2-500']

        ac3_63 = self.ac3['3-63']
        ac3_125 = self.ac3['3-125']
        ac3_250 = self.ac3['3-250']
        ac3_500 = self.ac3['3-500']

        ac4_63 = self.ac4['4-63']
        ac4_125 = self.ac4['4-125']
        ac4_250 = self.ac4['4-250']
        ac4_500 = self.ac4['4-500']

        ac5_63 = self.ac5['5-63']
        ac5_125 = self.ac5['5-125']
        ac5_250 = self.ac5['5-250']
        ac5_500 = self.ac5['5-500']

        ac6_63 = self.ac6['6-63']
        ac6_125 = self.ac6['6-125']
        ac6_250 = self.ac6['6-250']
        ac6_500 = self.ac6['6-500']

        # 63Hz dataset label
        [y_train_ac1_63,y_test_ac1_63] = np.split(ac1_63, [int(ac1_63.shape[0]*Ratio)], axis=0)
        [y_train_ac2_63,y_test_ac2_63] = np.split(ac2_63, [int(ac2_63.shape[0]*Ratio)], axis=0)
        [y_train_ac3_63,y_test_ac3_63] = np.split(ac3_63, [int(ac3_63.shape[0]*Ratio)], axis=0)
        [y_train_ac4_63,y_test_ac4_63] = np.split(ac4_63, [int(ac4_63.shape[0]*Ratio)], axis=0)
        [y_train_ac5_63,y_test_ac5_63] = np.split(ac5_63, [int(ac5_63.shape[0]*Ratio)], axis=0)
        [y_train_ac6_63,y_test_ac6_63] = np.split(ac6_63, [int(ac6_63.shape[0]*Ratio)], axis=0)
        y_train_ac1_63 =  y_train_ac1_63.values.astype("float32")
        y_test_ac1_63 =  y_test_ac1_63.values.astype("float32")
        y_train_ac2_63 =  y_train_ac2_63.values.astype("float32")
        y_test_ac2_63 =  y_test_ac2_63.values.astype("float32")
        y_train_ac3_63 =  y_train_ac3_63.values.astype("float32")
        y_test_ac3_63 =  y_test_ac3_63.values.astype("float32")
        y_train_ac4_63 =  y_train_ac4_63.values.astype("float32")
        y_test_ac4_63 =  y_test_ac4_63.values.astype("float32")
        y_train_ac5_63 =  y_train_ac5_63.values.astype("float32")
        y_test_ac5_63 =  y_test_ac5_63.values.astype("float32")
        y_train_ac6_63 =  y_train_ac6_63.values.astype("float32")
        y_test_ac6_63 =  y_test_ac6_63.values.astype("float32")


        # 125Hz dataset label
        [y_train_ac1_125,y_test_ac1_125] = np.split(ac1_125, [int(ac1_125.shape[0]*Ratio)], axis=0)
        [y_train_ac2_125,y_test_ac2_125] = np.split(ac2_125, [int(ac2_125.shape[0]*Ratio)], axis=0)
        [y_train_ac3_125,y_test_ac3_125] = np.split(ac3_125, [int(ac3_125.shape[0]*Ratio)], axis=0)
        [y_train_ac4_125,y_test_ac4_125] = np.split(ac4_125, [int(ac4_125.shape[0]*Ratio)], axis=0)
        [y_train_ac5_125,y_test_ac5_125] = np.split(ac5_125, [int(ac5_125.shape[0]*Ratio)], axis=0)
        [y_train_ac6_125,y_test_ac6_125] = np.split(ac6_125, [int(ac6_125.shape[0]*Ratio)], axis=0)
        y_train_ac1_125 =  y_train_ac1_125.values.astype("float32")
        y_test_ac1_125 =  y_test_ac1_125.values.astype("float32")
        y_train_ac2_125 =  y_train_ac2_125.values.astype("float32")
        y_test_ac2_125 =  y_test_ac2_125.values.astype("float32")
        y_train_ac3_125 =  y_train_ac3_125.values.astype("float32")
        y_test_ac3_125 =  y_test_ac3_125.values.astype("float32")
        y_train_ac4_125 =  y_train_ac4_125.values.astype("float32")
        y_test_ac4_125 =  y_test_ac4_125.values.astype("float32")
        y_train_ac5_125 =  y_train_ac5_125.values.astype("float32")
        y_test_ac5_125 =  y_test_ac5_125.values.astype("float32")
        y_train_ac6_125 =  y_train_ac6_125.values.astype("float32")
        y_test_ac6_125 =  y_test_ac6_125.values.astype("float32")

        # 250Hz dataset label
        [y_train_ac1_250,y_test_ac1_250] = np.split(ac1_250, [int(ac1_250.shape[0]*Ratio)], axis=0)
        [y_train_ac2_250,y_test_ac2_250] = np.split(ac2_250, [int(ac2_250.shape[0]*Ratio)], axis=0)
        [y_train_ac3_250,y_test_ac3_250] = np.split(ac3_250, [int(ac3_250.shape[0]*Ratio)], axis=0)
        [y_train_ac4_250,y_test_ac4_250] = np.split(ac4_250, [int(ac4_250.shape[0]*Ratio)], axis=0)
        [y_train_ac5_250,y_test_ac5_250] = np.split(ac5_250, [int(ac5_250.shape[0]*Ratio)], axis=0)
        [y_train_ac6_250,y_test_ac6_250] = np.split(ac6_250, [int(ac6_250.shape[0]*Ratio)], axis=0)
        y_train_ac1_250 =  y_train_ac1_250.values.astype("float32")
        y_test_ac1_250 =  y_test_ac1_250.values.astype("float32")
        y_train_ac2_250 =  y_train_ac2_250.values.astype("float32")
        y_test_ac2_250 =  y_test_ac2_250.values.astype("float32")
        y_train_ac3_250 =  y_train_ac3_250.values.astype("float32")
        y_test_ac3_250 =  y_test_ac3_250.values.astype("float32")
        y_train_ac4_250 =  y_train_ac4_250.values.astype("float32")
        y_test_ac4_250 =  y_test_ac4_250.values.astype("float32")
        y_train_ac5_250 =  y_train_ac5_250.values.astype("float32")
        y_test_ac5_250 =  y_test_ac5_250.values.astype("float32")
        y_train_ac6_250 =  y_train_ac6_250.values.astype("float32")
        y_test_ac6_250 =  y_test_ac6_250.values.astype("float32")

        # 500Hz dataset label
        [y_train_ac1_500,y_test_ac1_500] = np.split(ac1_500, [int(ac1_500.shape[0]*Ratio)], axis=0)
        [y_train_ac2_500,y_test_ac2_500] = np.split(ac2_500, [int(ac2_500.shape[0]*Ratio)], axis=0)
        [y_train_ac3_500,y_test_ac3_500] = np.split(ac3_500, [int(ac3_500.shape[0]*Ratio)], axis=0)
        [y_train_ac4_500,y_test_ac4_500] = np.split(ac4_500, [int(ac4_500.shape[0]*Ratio)], axis=0)
        [y_train_ac5_500,y_test_ac5_500] = np.split(ac5_500, [int(ac5_500.shape[0]*Ratio)], axis=0)
        [y_train_ac6_500,y_test_ac6_500] = np.split(ac6_500, [int(ac6_500.shape[0]*Ratio)], axis=0)
        y_train_ac1_500 =  y_train_ac1_500.values.astype("float32")
        y_test_ac1_500 =  y_test_ac1_500.values.astype("float32")
        y_train_ac2_500 =  y_train_ac2_500.values.astype("float32")
        y_test_ac2_500 =  y_test_ac2_500.values.astype("float32")
        y_train_ac3_500 =  y_train_ac3_500.values.astype("float32")
        y_test_ac3_500 =  y_test_ac3_500.values.astype("float32")
        y_train_ac4_500 =  y_train_ac4_500.values.astype("float32")
        y_test_ac4_500 =  y_test_ac4_500.values.astype("float32")
        y_train_ac5_500 =  y_train_ac5_500.values.astype("float32")
        y_test_ac5_500 =  y_test_ac5_500.values.astype("float32")
        y_train_ac6_500 =  y_train_ac6_500.values.astype("float32")
        y_test_ac6_500 =  y_test_ac6_500.values.astype("float32")

        # Room size
        [y_train_roomsize,y_test_roomsize] = np.split(self.RoomGeo, [int(self.RoomGeo.shape[0]*Ratio)], axis=0)
        y_train_roomsize =  y_train_roomsize.values.astype("float32")
        y_test_roomsize =  y_test_roomsize.values.astype("float32")

        ## Transfer function 
        [x_train,x_test] = np.split(self.TransFun, [int(self.TransFun.shape[0]*Ratio)], axis=0)
        x_train =  x_train.values
        x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
        x_test =  x_test.values
        x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)
        x_train = x_train.astype("float32")
        x_test = x_test.astype("float32")
        
        return x_train, x_test,y_train_ac1_63, y_train_ac1_125, y_train_ac1_250, y_train_ac1_500,\
                y_train_ac2_63, y_train_ac2_125, y_train_ac2_250, y_train_ac2_500,\
                    y_train_ac3_63, y_train_ac3_125, y_train_ac3_250, y_train_ac3_500,\
                y_train_ac4_63, y_train_ac4_125, y_train_ac4_250, y_train_ac4_500,\
                    y_train_ac5_63, y_train_ac5_125, y_train_ac5_250, y_train_ac5_500,\
                y_train_ac6_63, y_train_ac6_125, y_train_ac6_250, y_train_ac6_500,\
                    y_test_ac1_63, y_test_ac1_125, y_test_ac1_250, y_test_ac1_500,\
                y_test_ac2_63, y_test_ac2_125, y_test_ac2_250, y_test_ac2_500,\
                    y_test_ac3_63, y_test_ac3_125, y_test_ac3_250, y_test_ac3_500,\
                    y_test_ac4_63, y_test_ac4_125, y_test_ac4_250, y_test_ac4_500,\
                    y_test_ac5_63, y_test_ac5_125, y_test_ac5_250, y_test_ac5_500,\
                    y_test_ac6_63, y_test_ac6_125, y_test_ac6_250, y_test_ac6_500,y_train_roomsize, y_test_roomsize  

# Neural network class

In [ ]:
class ResNet(object):
    def __init__(self, length, num_channel, num_filters, room_size,pooling='avg', dropout_rate=False):
        self.length = length
        self.num_channel = num_channel
        self.num_filters = num_filters
        self.pooling = pooling
        self.dropout_rate = dropout_rate
        self.room_size = room_size
    # Neural network building blocks
    def Conv_1D_Block(self,x, num_filters, kernel, strides):
        # 1D Convolutional Block with BatchNormalization
        x = tf.keras.layers.Conv1D(num_filters, kernel, strides=strides, padding="same", kernel_initializer="he_normal")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        return x

    def conv_block(self,inputs, num_filters):
        # Construct Block of Convolutions without Pooling
        # x        : input into the block
        # n_filters: number of filters
        conv = self.Conv_1D_Block(inputs, num_filters, 3, 2)
        conv = self.Conv_1D_Block(conv, num_filters, 3, 1)

        return conv

    def residual_block(self,inputs, num_filters):
        # Construct a Residual Block of Convolutions
        # x        : input into the block
        # n_filters: number of filters
        shortcut = inputs
        conv = self.Conv_1D_Block(inputs, num_filters, 3, 1)
        conv = self.Conv_1D_Block(conv, num_filters, 3, 1)
        conv = tf.keras.layers.Add()([conv, shortcut])
        out = tf.keras.layers.Activation('relu')(conv)

        return out

    def residual_group(self,inputs, num_filters, n_blocks, conv=True):
        # x        : input to the group
        # n_filters: number of filters
        # n_blocks : number of blocks in the group
        # conv     : flag to include the convolution block connector
        out = inputs
        for i in range(n_blocks):
            out = self.residual_block(out, num_filters)

        # Double the size of filters and reduce feature maps by 75% (strides=2, 2) to fit the next Residual Group
        if conv:
            out = self.conv_block(out, num_filters * 2)
        return out


    def learner18(self,inputs, num_filters):
        # Construct the Learner
        block1 = self.residual_group(inputs, num_filters, 2)          # First Residual Block Group of 64 filters
        block2 = self.residual_group(block1, num_filters * 2, 1)           # Second Residual Block Group of 128 filters
        block3 = self.residual_group(block2, num_filters * 4, 1)           # Third Residual Block Group of 256 filters

        return block3

    def stem(self,inputs, num_filters):
        # Construct the Stem Convolution Group
        # inputs : input vector
        # First Convolutional layer, where pooled feature maps will be reduced by 75%
        # In the input layer, the kernel size is 7 and strides is 2, so the 354 reduced to 177 after this filter
        # Maxpooling. pool size = 2 means it will choose 2 element, and move by 2
        conv =self.Conv_1D_Block(inputs, num_filters, 7, 2)
        if conv.shape[1] <= 2:
            print('1')
            pool = tf.keras.layers.MaxPooling1D(pool_size=1, strides=2, padding="valid")(conv)
        else:
            pool = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2, padding="valid")(conv)
        return pool

    def task(self, x):
        # 63Hz frequency band branch
        main_63 = self.residual_group(x, self.num_filters*8 , 1, False)  # Fourth Residual Block Group of 512 filters
        main_63 = tf.keras.layers.GlobalAveragePooling1D()(main_63)
        main_63 = tf.keras.layers.Dense(1000, activation='relu')(main_63)
        
        # 125Hz frequency band branch
        main_125 = self.residual_group(x, self.num_filters*8 , 1, False)  # Fourth Residual Block Group of 512 filters
        main_125 = tf.keras.layers.GlobalAveragePooling1D()(main_125)
        main_125 = tf.keras.layers.Dense(1000, activation='relu')(main_125)
        
        # 250Hz frequency band branch
        main_250 = self.residual_group(x, self.num_filters*8 , 1, False)  # Fourth Residual Block Group of 512 filters
        main_250 = tf.keras.layers.GlobalAveragePooling1D()(main_250)
        main_250 = tf.keras.layers.Dense(1000, activation='relu')(main_250) 
        
        # 500Hz frequency band branch
        main_500 = self.residual_group(x, self.num_filters*8 , 1, False)  # Fourth Residual Block Group of 512 filters
        main_500 = tf.keras.layers.GlobalAveragePooling1D()(main_500)
        main_500 = tf.keras.layers.Dense(1000, activation='relu')(main_500) 
        
        # Surface 1 branch
        ac1_63_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac1_63_output')(main_63)
        ac1_125_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac1_125_output')(main_125)
        ac1_250_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac1_250_output')(main_250)
        ac1_500_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac1_500_output')(main_500)
        
        # Surface 2 branch
        ac2_63_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac2_63_output')(main_63)
        ac2_125_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac2_125_output')(main_125)
        ac2_250_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac2_250_output')(main_250)     
        ac2_500_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac2_500_output')(main_500)     

        # Surface 3 branch
        ac3_63_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac3_63_output')(main_63)
        ac3_125_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac3_125_output')(main_125)
        ac3_250_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac3_250_output')(main_250)
        ac3_500_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac3_500_output')(main_500)

        # Surface 4 branch
        ac4_63_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac4_63_output')(main_63)
        ac4_125_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac4_125_output')(main_125)
        ac4_250_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac4_250_output')(main_250)
        ac4_500_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac4_500_output')(main_500)

        # Surface 5 branch
        ac5_63_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac5_63_output')(main_63)
        ac5_125_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac5_125_output')(main_125)
        ac5_250_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac5_250_output')(main_250)
        ac5_500_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac5_500_output')(main_500)

        # Surface 6 branch
        ac6_63_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac6_63_output')(main_63)
        ac6_125_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac6_125_output')(main_125)
        ac6_250_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac6_250_output')(main_250)
        ac6_500_branch = tf.keras.layers.Dense(1, activation='sigmoid', name='ac6_500_output')(main_500)
        
        # Weather to include room size branch
        if self.room_size:
            room_size_branch = tf.keras.layers.Dense(3, activation='relu', name='roomsize_output')(main_63)
            outputs = [ac1_63_branch,ac2_63_branch,ac3_63_branch,ac4_63_branch,ac5_63_branch,ac6_63_branch,ac1_125_branch,ac2_125_branch,ac3_125_branch,ac4_125_branch,ac5_125_branch,ac6_125_branch,ac1_250_branch,ac2_250_branch,ac3_250_branch,ac4_250_branch,ac5_250_branch,ac6_250_branch,ac1_500_branch,ac2_500_branch,ac3_500_branch,ac4_500_branch,ac5_500_branch,ac6_500_branch,room_size_branch]
        else:
            outputs = [ac1_63_branch,ac2_63_branch,ac3_63_branch,ac4_63_branch,ac5_63_branch,ac6_63_branch,ac1_125_branch,ac2_125_branch,ac3_125_branch,ac4_125_branch,ac5_125_branch,ac6_125_branch,ac1_250_branch,ac2_250_branch,ac3_250_branch,ac4_250_branch,ac5_250_branch,ac6_250_branch,ac1_500_branch,ac2_500_branch,ac3_500_branch,ac4_500_branch,ac5_500_branch,ac6_500_branch]
        return outputs

    def ResNet18(self):
        # Model sequence
        inputs = tf.keras.Input((self.length, self.num_channel), name='main_input')      # The input tensor
        stem_ = self.stem(inputs, self.num_filters)               # The Stem Convolution Group
        x = self.learner18(stem_, self.num_filters)               # The learner
        outputs = self.task(x)
        
        # Instantiate the Model
        model = tf.keras.Model(inputs, outputs)
        return model

# Load data, instantiate neural network and define optimizer

In [ ]:
# 1. Load data
path = 'dataset/'  # dataset path
ratio = 0.8        # ratio between training and testing data set
# Instantiate dataset
data = DataLoader(path,ratio)
x_train, x_test,y_train_ac1_63, y_train_ac1_125, y_train_ac1_250, y_train_ac1_500,\
                y_train_ac2_63, y_train_ac2_125, y_train_ac2_250, y_train_ac2_500,\
                    y_train_ac3_63, y_train_ac3_125, y_train_ac3_250, y_train_ac3_500,\
                y_train_ac4_63, y_train_ac4_125, y_train_ac4_250, y_train_ac4_500,\
                    y_train_ac5_63, y_train_ac5_125, y_train_ac5_250, y_train_ac5_500,\
                y_train_ac6_63, y_train_ac6_125, y_train_ac6_250, y_train_ac6_500,\
                    y_test_ac1_63, y_test_ac1_125, y_test_ac1_250, y_test_ac1_500,\
                y_test_ac2_63, y_test_ac2_125, y_test_ac2_250, y_test_ac2_500,\
                    y_test_ac3_63, y_test_ac3_125, y_test_ac3_250, y_test_ac3_500,\
                    y_test_ac4_63, y_test_ac4_125, y_test_ac4_250, y_test_ac4_500,\
                    y_test_ac5_63, y_test_ac5_125, y_test_ac5_250, y_test_ac5_500,\
                    y_test_ac6_63, y_test_ac6_125, y_test_ac6_250, y_test_ac6_500,y_train_roomsize, y_test_roomsize=data.load_data()



# 2. Instaitiate the network
length = x_train.shape[1]   # Number of Features (or length of the signal)
num_filters = 4           # Number of Filter  in the Input Layer
num_channel = 1             # Number of Input Channels
room_size = True           # weather to add room size estimation
model = ResNet(length, num_channel,num_filters,room_size).ResNet18() # Build Model

# 3. Define optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
loss={'ac1_63_output': 'mse', 'ac1_125_output': 'mse', 'ac1_250_output': 'mse',
                                     'ac1_500_output': 'mse',
                                     'ac2_63_output': 'mse', 'ac2_125_output': 'mse', 'ac2_250_output': 'mse',
                                     'ac2_500_output': 'mse',
                                     'ac3_63_output': 'mse', 'ac3_125_output': 'mse', 'ac3_250_output': 'mse',
                                     'ac3_500_output': 'mse',
                                     'ac4_63_output': 'mse', 'ac4_125_output': 'mse', 'ac4_250_output': 'mse',
                                     'ac4_500_output': 'mse',
                                     'ac5_63_output': 'mse', 'ac5_125_output': 'mse', 'ac5_250_output': 'mse',
                                     'ac5_500_output': 'mse',
                                     'ac6_63_output': 'mse', 'ac6_125_output': 'mse', 'ac6_250_output': 'mse',
                                     'ac6_500_output': 'mse',
                                     'roomsize_output': 'mse'},
                               loss_weights={'ac1_63_output': 1, 'ac1_125_output': 1, 'ac1_250_output': 1,
                                             'ac1_500_output': 1,
                                             'ac2_63_output': 1, 'ac2_125_output': 1, 'ac2_250_output': 1,
                                             'ac2_500_output': 1,
                                             'ac3_63_output': 1, 'ac3_125_output': 1, 'ac3_250_output': 1,
                                             'ac3_500_output': 1,
                                             'ac4_63_output': 1, 'ac4_125_output': 1, 'ac4_250_output': 1,
                                             'ac4_500_output': 1,
                                             'ac5_63_output': 1, 'ac5_125_output': 1, 'ac5_250_output': 1,
                                             'ac5_500_output': 1,
                                             'ac6_63_output': 1, 'ac6_125_output': 1, 'ac6_250_output': 1,
                                             'ac6_500_output': 1,'roomsize_output': 1})

# Trainning

In [ ]:
plot_progress = PlotProgress(entity='loss')
Epoch = 200
BatchSize = 64

try:
    history = model.fit({'main_input': x_train},
                                     {'ac1_63_output': y_train_ac1_63, 'ac1_125_output': y_train_ac1_125,
                                      'ac1_250_output': y_train_ac1_250, 'ac1_500_output': y_train_ac1_500,
                                      'ac2_63_output': y_train_ac2_63, 'ac2_125_output': y_train_ac2_125,
                                      'ac2_250_output': y_train_ac2_250, 'ac2_500_output': y_train_ac2_500,
                                      'ac3_63_output': y_train_ac3_63, 'ac3_125_output': y_train_ac3_125,
                                      'ac3_250_output': y_train_ac3_250, 'ac3_500_output': y_train_ac3_500,
                                      'ac4_63_output': y_train_ac4_63, 'ac4_125_output': y_train_ac4_125,
                                      'ac4_250_output': y_train_ac4_250, 'ac4_500_output': y_train_ac4_500,
                                      'ac5_63_output': y_train_ac5_63, 'ac5_125_output': y_train_ac5_125,
                                      'ac5_250_output': y_train_ac5_250, 'ac5_500_output': y_train_ac5_500,
                                      'ac6_63_output': y_train_ac6_63, 'ac6_125_output': y_train_ac6_125,
                                      'ac6_250_output': y_train_ac6_250, 'ac6_500_output': y_train_ac6_500,
                                      'roomsize_output': y_train_roomsize},
              epochs=Epoch, batch_size=BatchSize,
              verbose=1,
              shuffle=True,
              callbacks=[plot_progress],
              validation_split=0.2,
             )
except KeyboardInterrupt:
    pass

## Save training results

In [ ]:
Item = ['Roomsize', '1-63', '1-125', '1-250', '1-500', '2-63', '2-125', '2-250', '2-500', '3-63', '3-125',
                 '3-250', '3-500', '4-63',
                 '4-125', '4-250', '4-500', '5-63', '5-125', '5-250', '5-500', '6-63', '6-125', '6-250', '6-500']
EpochNumber = []
for i in range(Epoch):
    EpochNumber.append(str(i+1))

# Save trainning loss
TrainList = [history.history['roomsize_output_loss'], history.history['ac1_63_output_loss'],
             history.history['ac1_125_output_loss'], history.history['ac1_250_output_loss'],
             history.history['ac1_500_output_loss'],
             history.history['ac2_63_output_loss'], history.history['ac2_125_output_loss'],
             history.history['ac2_250_output_loss'], history.history['ac2_500_output_loss'],
             history.history['ac3_63_output_loss'],
             history.history['ac3_125_output_loss'], history.history['ac3_250_output_loss'],
             history.history['ac3_500_output_loss'],
             history.history['ac4_63_output_loss'], history.history['ac4_125_output_loss'],
             history.history['ac4_250_output_loss'], history.history['ac4_500_output_loss'],
             history.history['ac5_63_output_loss'],
             history.history['ac5_125_output_loss'], history.history['ac5_250_output_loss'],
             history.history['ac5_500_output_loss'],
             history.history['ac6_63_output_loss'], history.history['ac6_125_output_loss'],
             history.history['ac6_250_output_loss'], history.history['ac6_500_output_loss']]
TrainLoss = pd.DataFrame(columns=EpochNumber, index=Item, data=TrainList)
TrainLoss.to_csv('TrainLosslog.csv', encoding='utf-8')

# Save testing loss
ValList = [history.history['val_roomsize_output_loss'], history.history['val_ac1_63_output_loss'],
            history.history['val_ac1_125_output_loss'], history.history['val_ac1_250_output_loss'],
            history.history['val_ac1_500_output_loss'],
            history.history['val_ac2_63_output_loss'], history.history['val_ac2_125_output_loss'],
            history.history['val_ac2_250_output_loss'], history.history['val_ac2_500_output_loss'],
            history.history['val_ac3_63_output_loss'],
            history.history['val_ac3_125_output_loss'], history.history['val_ac3_250_output_loss'],
            history.history['val_ac3_500_output_loss'],
            history.history['val_ac4_63_output_loss'], history.history['val_ac4_125_output_loss'],
            history.history['val_ac4_250_output_loss'], history.history['val_ac4_500_output_loss'],
            history.history['val_ac5_63_output_loss'],
            history.history['val_ac5_125_output_loss'], history.history['val_ac5_250_output_loss'],
            history.history['val_ac5_500_output_loss'],
            history.history['val_ac6_63_output_loss'], history.history['val_ac6_125_output_loss'],
            history.history['val_ac6_250_output_loss'], history.history['val_ac6_500_output_loss']]
ValLoss = pd.DataFrame(columns=EpochNumber, index=Item, data=TestList);
ValLoss.to_csv('ValLosslog.csv', encoding='utf-8')

# Testing

In [ ]:
print("Evaluate model on test data")
results =model.evaluate(x_test,
[y_test_ac1_63, y_test_ac2_63, y_test_ac3_63, y_test_ac4_63, y_test_ac5_63,
y_test_ac6_63,
y_test_ac1_125, y_test_ac2_125, y_test_ac3_125, y_test_ac4_125,
y_test_ac5_125, y_test_ac6_125,
y_test_ac1_250, y_test_ac2_250, y_test_ac3_250, y_test_ac4_250,
y_test_ac5_250, y_test_ac6_250,
y_test_ac1_500, y_test_ac2_500, y_test_ac3_500, y_test_ac4_500,
y_test_ac5_500, y_test_ac6_500, y_test_roomsize], batch_size=32)


In [ ]:
LossTerm=['OverallLoss', '1-63', '2-63', '3-63', '4-63', '5-63', '6-63', '1-125', '2-125', '3-125', '4-125',
                 '5-125', '6-125', '1-250', '2-250', '3-250', '4-250', '5-250', '6-250', '1-500', '2-500', '3-500',
                 '4-500', '5-500', '6-500', 'Roomsize']
EvaluateLoss=pd.DataFrame(index=LossTerm,data=results)
EvaluateLoss.to_csv('EvaluateLoss.csv',encoding='utf-8')

# Save model

In [ ]:
model.save('model.h5')